In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import re
import pandas as pd

#scrolling function
def scroll_to_bottom(driver):
    scrolling = True
    while scrolling:
        old_page_source = driver.page_source
        body=driver.find_element(By.TAG_NAME, 'body')
        body.send_keys(Keys.END)
        time.sleep(3)
        new_page_source = driver.page_source
        if old_page_source == new_page_source:
            scrolling = False

driver = webdriver.Chrome()
wait = WebDriverWait(driver, 20)

driver.get("https://www.redbus.in/online-booking/astc");
driver.maximize_window()
scroll_to_bottom(driver)


#to check viewbus button for gvt buses
def click_view_buses_buttons(driver, wait):
    while True:
        try:
            # Find all "View Buses" buttons
            view_buses_buttons = driver.find_elements(By.XPATH, '//div[@class="clearfix"]//div[contains(@class, "button") and contains(text(), "View Buses")]')
            if not view_buses_buttons:
                break

            for view_buses_button in view_buses_buttons:
                try:
                    # Scroll the button into view 
                    actions = ActionChains(driver)
                    actions.move_to_element(view_buses_button).perform()
                    time.sleep(2)

                    # Click the button
                    view_buses_button.click()
                    wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'travels')))
                    time.sleep(3)
                except Exception as e:
                    print("Error at clicking view bus button:",e)
        except Exception as e:
            print("Error at finding view bus button:",e)
            break

#extracting all bus details
def extract_all_bus_items(driver, wait, route_link, route_title):
    driver.get(route_link)
    wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'travels')))
    time.sleep(2)

    scroll_to_bottom(driver)
    click_view_buses_buttons(driver, wait)

    bus_items = []
    bus_elements = driver.find_elements(By.CLASS_NAME, 'clearfix.bus-item')
    for bus in bus_elements:
        try:
            # Seat availability text
            seat_left_text = bus.find_element(By.CLASS_NAME, 'seat-left').text
                
            # Regular expression to find the first integer in the text
            match = re.search(r'\d+', seat_left_text)
            seat_left = int(match.group(0)) if match else 0
            
            details = {
               'route_name': route_title,
                'route_link': route_link,
                'busname': '',
                'bustype': '',
                'departing_time': '',
                'departing_location': '',
                'duration': '',
                'reaching_time': '',
                'boardingpoint_location': '',
                'star_rating': '',
                'price': '',
                'seats_available': ''
            }

            try:
                details['busname'] = bus.find_element(By.CLASS_NAME, 'travels').text
            except Exception as e:
                print("Error at extracting busname:",e)

            try:
                details['bustype'] = bus.find_element(By.CLASS_NAME, 'bus-type').text
            except Exception as e:
                print("Error at extracting bustype:",e)

            try:
                details['departing_time'] = bus.find_element(By.CLASS_NAME, 'dp-time').text
            except Exception as e:
                print("Error at extracting departing time:",e)

            try:
                details['departing_location'] = bus.find_element(By.CLASS_NAME, 'dp-loc').get_attribute('title')
            except Exception as e:
                print("Error at extracting departing location:",e)

            try:
                details['duration'] = bus.find_element(By.CLASS_NAME, 'dur').text
            except Exception as e:
                print("Error at extracting duration:",e)

            try:
                details['reaching_time'] = bus.find_element(By.CLASS_NAME, 'bp-time').text
            except Exception as e:
                print("Error at extracting reaching time:",e)

            try:
                details['boardingpoint_location'] = bus.find_element(By.CLASS_NAME, 'bp-loc').get_attribute('title')
            except Exception as e:
                print("Error at extracting boarding point location:",e)

            try:
                details['star_rating'] = bus.find_element(By.CLASS_NAME, 'rating').text
            except Exception as e:
                print("Error at extracting star rating:",e)

            try:
                details['price'] =  int(bus.find_element(By.CSS_SELECTOR, '.seat-fare .fare.d-block .f-19.f-bold').text)
            except Exception as e:
                print("Error at extracting price:",e)

            try:
                details['seats_available'] = seat_left
            except Exception as e:
                print("Error at extracting seats available:",e)

            bus_items.append(details)
            print(f"Loaded bus details: {details}")
        except Exception as e:
            print(f"Error extracting bus details: {e}")
            continue
    return bus_items

#find routename and route link
all_routes = []
while True:
    try:
        ktcl_routes = driver.find_elements(By.CLASS_NAME, "route")
        for route in ktcl_routes:
            title = route.text
            link = route.get_attribute('href')
            if title and link:
                all_routes.append({"title": title, "link": link})

        pagination_container = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "DC_117_paginationTable")))
        current_page = pagination_container.find_element(By.CLASS_NAME, "DC_117_pageActive").text
        next_page_number = int(current_page) + 1
        next_page_button = pagination_container.find_element(By.XPATH, f'.//div[contains(@class, "DC_117_pageTabs") and text()="{next_page_number}"]')
        actions = ActionChains(driver)
        actions.move_to_element(next_page_button).perform()
        time.sleep(1)
        next_page_button.click()
        wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "route")))
        time.sleep(2)
    except Exception as e:
        break

all_bus_details = []

print("Loading,...")

try:
    for route in all_routes:
        route_title = route['title']
        route_link = route['link']
        try:
            bus_items = extract_all_bus_items(driver, wait, route_link, route_title)
            all_bus_details.extend(bus_items)
        except Exception as e:
            print("Error at extracting route and link with bus details;",e)
except Exception as e:
    print("Error:",e)
finally:
    driver.quit()

df= pd.DataFrame(all_bus_details)

print("\nBus Details done:")


Loading,...
Loaded bus details: {'route_name': 'Tezpur to Guwahati', 'route_link': 'https://www.redbus.in/bus-tickets/tezpur-to-guwahati', 'busname': 'Assam State Transport Corporation (ASTC) - 154930', 'bustype': 'Volvo AC Seater 2+2', 'departing_time': '15:45', 'departing_location': 'ASTC Bus Stand Tezpur', 'duration': '04h 15m', 'reaching_time': '20:00', 'boardingpoint_location': 'Khanapara', 'star_rating': '4.5', 'price': 298, 'seats_available': 23}
Loaded bus details: {'route_name': 'Tezpur to Guwahati', 'route_link': 'https://www.redbus.in/bus-tickets/tezpur-to-guwahati', 'busname': 'Assam State Transport Corporation (ASTC) - 161055', 'bustype': 'Volvo AC Seater 2+2', 'departing_time': '16:15', 'departing_location': 'ASTC Bus Stand Tezpur', 'duration': '04h 15m', 'reaching_time': '20:30', 'boardingpoint_location': 'ASTC Paltan Bazar', 'star_rating': '3.9', 'price': 298, 'seats_available': 22}
Error at extracting price: invalid literal for int() with base 10: '389.13'
Loaded bus d

In [2]:
df

,route_name,route_link,busname,bustype,departing_time,departing_location,duration,reaching_time,boardingpoint_location,star_rating,price,seats_available
0,Tezpur to Guwahati,https://www.redbus.in/bus-tickets/tezpur-to-gu...,Assam State Transport Corporation (ASTC) - 154930,Volvo AC Seater 2+2,15:45,ASTC Bus Stand Tezpur,04h 15m,20:00,Khanapara,4.5,298,23
1,Tezpur to Guwahati,https://www.redbus.in/bus-tickets/tezpur-to-gu...,Assam State Transport Corporation (ASTC) - 161055,Volvo AC Seater 2+2,16:15,ASTC Bus Stand Tezpur,04h 15m,20:30,ASTC Paltan Bazar,3.9,298,22
2,Tezpur to Guwahati,https://www.redbus.in/bus-tickets/tezpur-to-gu...,Chartered Bus - ASTC,Volvo AC Seater Pushback 2+2,13:30,Porua Chariali,03h 30m,17:00,Khanapara,4.7,,14
3,Tezpur to Guwahati,https://www.redbus.in/bus-tickets/tezpur-to-gu...,Chartered Bus - ASTC,Volvo AC Seater Pushback 2+2,14:30,Tezpur Astc,04h 00m,18:30,ISBT Guwahati,4.8,,27
4,Tezpur to Guwahati,https://www.redbus.in/bus-tickets/tezpur-to-gu...,SHIVAM TRAVELS,Bharat Benz A/C Seater (2+1),15:00,Tezpur Bus Stop,04h 15m,19:15,Khanapara,4.2,360,32
...,...,...,...,...,...,...,...,...,...,...,...,...
167,Tezpur to Tinsukia,https://www.redbus.in/bus-tickets/tezpur-to-ti...,Neha Travels,NON A/C Sleeper (2+1),22:40,Tezpur (Mission Chariali,09h 20m,08:00,Tinsukia Astc Bus Stand,1.4,650,21
168,Dibrugarh to Bokakhat,https://www.redbus.in/bus-tickets/dibrugarh-to...,Baikuntha Transport Service,Bharat Benz A/C Seater /Sleeper (2+1),19:30,University Bus Stop (Dibrugarh,05h 30m,01:00,BOKAKHAT ASTC,3.9,500,40
169,Dibrugarh to Bokakhat,https://www.redbus.in/bus-tickets/dibrugarh-to...,Purple Wings Coaches,A/C Seater / Sleeper (2+1),19:55,AMC Bus Stop,05h 20m,01:15,Bokakhat (A.S.T.C,3.5,569,41
170,Dibrugarh to Bokakhat,https://www.redbus.in/bus-tickets/dibrugarh-to...,Shree Ram Travels,Bharat Benz A/C Seater /Sleeper (2+1),20:45,ASTC Chowkidingi Bus Stand,04h 55m,01:40,Bokakhat bus stand,4.8,530,12


In [3]:
#convert dataframe into csv
path = "D:\\CHITRA\\REDBUSPROJECT\\rb_assam.csv"
df.to_csv(path, index=False)

In [4]:
#connect python to mysql

In [5]:
import pandas as pd
import pymysql
import re

#clean data
def clean_data(value, data_type):
    if pd.isna(value):  # Handle NaN values
        return None
    if data_type == 'int':
        return int(''.join(filter(str.isdigit, str(value))))   # example: a="123,4" str()='1','2','3','4', to 1234
    elif data_type == 'float':
        return float(''.join(filter(lambda x: x.isdigit() or x == '.', str(value))))
    else:
        return value

#convert duration to 'HH:MM:SS' format
def convert_duration(duration):
    if pd.isna(duration):  # Handle NaN values
        return None
    match = re.match(r'(\d+)h\s*(\d+)m', duration)  # s* for replace space
    if match:
        hours = int(match.group(1))
        minutes = int(match.group(2))
        return f'{hours:02}:{minutes:02}:00'    # like 02:30:00
    return None  # return None if format does not match

# Read CSV file
csv_file_path = r"D:\\CHITRA\\REDBUSPROJECT\\rb_assam.csv"
df = pd.read_csv(csv_file_path)

# Connect to MySQL
connection = pymysql.connect(
    host='127.0.0.1',
    user='root',
    password='2210',
    database='finalredbusproject'
)
cursor = connection.cursor()

# Create table
create_table_query = """
CREATE TABLE IF NOT EXISTS rb_assam (
    id INT AUTO_INCREMENT PRIMARY KEY,
    route_name VARCHAR(255),
    route_link VARCHAR(255),
    busname VARCHAR(255),
    bustype VARCHAR(255),
    departing_time TIME,
    duration TIME,
    reaching_time TIME,
    star_rating FLOAT,
    price INT,
    seats_available INT
);
"""
cursor.execute(create_table_query)

# all NaNs (not a number) are replaced with None
df = df.where(pd.notnull(df), None)

# Insert the extracted rows into the MySQL table...
# ensuring that all data from the DataFrame is inserted into the database table row by row.
for index, row in df.iterrows():  # df.iterrows each row
    insert_query = """
    INSERT INTO rb_assam (route_name, route_link, busname, bustype, departing_time, duration, reaching_time, star_rating, price, seats_available)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    # %s is a placeholder for each column
    row_data = (
        row['route_name'],
        row['route_link'],
        row['busname'],
        row['bustype'],
        row['departing_time'],
        convert_duration(row['duration']),
        row['reaching_time'],
        clean_data(row['star_rating'], 'float'),
        clean_data(row['price'], 'int'),
        clean_data(row['seats_available'], 'int')
    )
    
    try:
        cursor.execute(insert_query, row_data)
    except Exception as e:
        print(f"Error inserting row {index}: {e}")
        print(f"Row data: {row_data}")

# Commit and close the connection
connection.commit()
cursor.close()
connection.close()

print("Data inserted successfully!")


Data inserted successfully!
